# 用scikit-learn,gensim计算中文文章的tfidf
@author: humch <br>
@date: 2018-3-18

关于tf-idf的计算公式及意义，这边不再说明，网上有很多相关的资料，这边只谈在python中如何调用第三包**计算中文文本的tfidf**。


## scikit-learn

#### 处理英文
scikit-learn是用Python开发的机器学习库,对于计算tfidf网上给的例子如下

In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

In [2]:
corpus_en = [
    'This is the first document.',
    'This is the second second document.',
    'And the third one.',
    'Is this the first document?',
]
vectorizer = CountVectorizer()
count = vectorizer.fit_transform(corpus_en)
print(vectorizer.get_feature_names())
print(vectorizer.vocabulary_)
print(count.toarray())

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']
{'the': 6, 'this': 8, 'is': 3, 'first': 2, 'and': 0, 'one': 4, 'third': 7, 'document': 1, 'second': 5}
[[0 1 1 1 0 0 1 0 1]
 [0 1 0 1 0 2 1 0 1]
 [1 0 0 0 1 0 1 1 0]
 [0 1 1 1 0 0 1 0 1]]


#### 处理中文

从上面的的例子我们可以看到，corpus一个英文字符串列表，分词(显然是用空格分割)和其他预处理(如去标点，过滤小于两个字符的单词)是在CountVectorizer类的内部进行的。很容易地，在处理中文时，我们会想到先将文本在外部进行分词过滤，然后再用空格拼接成字符串形成corpus,然后再按上面的方法得到中文文档的tf-idf

In [3]:
import jieba
import re

In [4]:
corpus_zh = ["我来到北京清华大学",
        "他来到了网易杭研大厦",
        "小明硕士毕业于中国科学院",
        "我爱北京天安门！！"]
biaodian = ("！", "。", "？")

## 结巴分词，去标点，再用空格拼接
corpus_zh_new = [" ".join([w for w in jieba.cut(s) if w not in biaodian]) for s in corpus_zh]
print(corpus_zh_new)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.822 seconds.
Prefix dict has been built succesfully.


['我 来到 北京 清华大学', '他 来到 了 网易 杭研 大厦', '小明 硕士 毕业 于 中国科学院', '我 爱 北京 天安门']


In [5]:
## 接下来和上面的英文文档的处理方法一样
vectorizer = CountVectorizer()
transformer = TfidfTransformer()
tfidf = transformer.fit_transform(vectorizer.fit_transform(corpus_zh_new))
print(vectorizer.get_feature_names())
print("--------------")
print(vectorizer.vocabulary_)
print("--------------")
print(tfidf.toarray())

['中国科学院', '北京', '大厦', '天安门', '小明', '来到', '杭研', '毕业', '清华大学', '硕士', '网易']
--------------
{'北京': 1, '毕业': 7, '天安门': 3, '大厦': 2, '来到': 5, '硕士': 9, '杭研': 6, '网易': 10, '中国科学院': 0, '清华大学': 8, '小明': 4}
--------------
[[ 0.          0.52640543  0.          0.          0.          0.52640543
   0.          0.          0.66767854  0.          0.        ]
 [ 0.          0.          0.52547275  0.          0.          0.41428875
   0.52547275  0.          0.          0.          0.52547275]
 [ 0.5         0.          0.          0.          0.5         0.          0.
   0.5         0.          0.5         0.        ]
 [ 0.          0.6191303   0.          0.78528828  0.          0.          0.
   0.          0.          0.          0.        ]]


#### 分析源码

在上面的操作中，我们实现了中文文档的tfidf计算，不过出于完美主义，总是觉得不够直接，有额外多余的步骤，而且量一上来，每个文档都要再用空格拼接，然后在CountVectorizer类里再分割，整体速度明显会受到影响。所以是不是有和处理英文一样方式来处理中文呢？我们来看翻源代码<a href="https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/feature_extraction/text.py">sklearn.feature_extraction.text</a> <br>
可以看到，这个文件里有一下几个类：<br>
- VectorizerMixin ：文本矢量化(text vectorizers)类的公共代码部分
- HashingVectorizer ：继承了BaseEstimator, VectorizerMixin, TransformerMixin三个类，将文档转换成矩阵
- CountVectorizer ：统计向量器，将文档集合转换成统计切词的矩阵， 继承了BaseEstimator, VectorizerMixin
- TfidfTransformer ：Tfidf转换器，将统计矩阵转换成标准化的tf或tf-idf，继承了BaseEstimator和TransformerMixin
- TfidfVectorizer ：tf-idf向量器，将原文本(未分过词的字符传)转换成tf-idf特征矩阵，继承自CountVectorizer

根据之前的代码，我们关心的是CountVectorizer类，所以我们来看看这个类的fit_transform方法。<br>
fit_transform方法输入参数为raw_documents, raw_documents为可迭代对象，每次迭代返回字符串文档。通过追溯fit_transform的调用，我们可以看到，raw_documents会在_cont_vocab方法里被迭代处理，进行统计词频等一系列转换成矩阵的操作：

```
def _count_vocab(self, raw_documents, fixed_vocab):
    ...
    analyze = self.build_analyzer()
    j_indices = []
    indptr = []

    values = _make_int_array()
    indptr.append(0)
    for doc in raw_documents:
        feature_counter = {}
        for feature in analyze(doc):
            try:
                feature_idx = vocabulary[feature]
                if feature_idx not in feature_counter:
                    feature_counter[feature_idx] = 1
                else:
                    feature_counter[feature_idx] += 1
            except KeyError:
                continue
    ...
```

迭代每条document过程中，由analyze处理每条document生成特征，analyze由**build_analyzer方法(划重点)**生成：

```
def build_analyzer(self):
    """Return a callable that handles preprocessing and tokenization"""
    if callable(self.analyzer):
        return self.analyzer

    preprocess = self.build_preprocessor()

    if self.analyzer == 'char':
        return lambda doc: self._char_ngrams(preprocess(self.decode(doc)))

    elif self.analyzer == 'char_wb':
        return lambda doc: self._char_wb_ngrams(
            preprocess(self.decode(doc)))

    elif self.analyzer == 'word':
        stop_words = self.get_stop_words()
        tokenize = self.build_tokenizer()

        return lambda doc: self._word_ngrams(
            tokenize(preprocess(self.decode(doc))), stop_words)

    else:
        raise ValueError('%s is not a valid tokenization scheme/analyzer' %
                         self.analyzer)
```

这边涉及到CountVectorizer类初始化实例时相关的几个参数:<br>
- analyzer: 参数的可能取值见上面那段代码
- preprocess: 文档预处理，
- tokenizer: 分词器，输入字符串文档，迭代返回(yield)词
- ngram_range
- stop_words
- lowcase
- token_pattern: 是个正则表达式，在tokenizer是None时才会用到

从build_analyzer方法里的这段代码，我们可以很清楚的看到，**在初始化过程中，如果analyzer的值是个可调用的函数，那么其余的参数(preprocess,stop_words,tokenizer等)将不起作用。**如果只是处理中文，那么我们到这里就可以结束了，我们只要定义一个处理每条文档的函数，该函数输入字符串文档，迭代返回特征词，函数里面可以加入分词，过滤等我们需要的功能，然后在初始化时传入这个函数，就ok了。

In [6]:
def analyzer(doc):
    """这边可以根据需要做更多的处理，如去停用词
    """
    for token in jieba.cut(doc):
        if token not in biaodian and len(token) > 1:
            yield token

vectorizer=CountVectorizer(analyzer=analyzer)
transformer = TfidfTransformer()
tfidf = transformer.fit_transform(vectorizer.fit_transform(corpus_zh))

print(vectorizer.get_feature_names())
print("--------------")
print(vectorizer.vocabulary_)
print("--------------")
print(tfidf.toarray())


['中国科学院', '北京', '大厦', '天安门', '小明', '来到', '杭研', '毕业', '清华大学', '硕士', '网易']
--------------
{'北京': 1, '毕业': 7, '天安门': 3, '大厦': 2, '来到': 5, '硕士': 9, '杭研': 6, '网易': 10, '中国科学院': 0, '清华大学': 8, '小明': 4}
--------------
[[ 0.          0.52640543  0.          0.          0.          0.52640543
   0.          0.          0.66767854  0.          0.        ]
 [ 0.          0.          0.52547275  0.          0.          0.41428875
   0.52547275  0.          0.          0.          0.52547275]
 [ 0.5         0.          0.          0.          0.5         0.          0.
   0.5         0.          0.5         0.        ]
 [ 0.          0.6191303   0.          0.78528828  0.          0.          0.
   0.          0.          0.          0.        ]]


当然，因为analyzer的默认值是words,观察build_analyzer函数，我们可以得知，也可以在初始化时传入tokenizer一个可调用函数，函数功能同上面analyzer，同时可以传入传入stop_words和preprocess参数，所以如果你要用到CountVectorizer初始化时其他参数的功能(如ngram-range),那么你可以实现一个tokenizer函数，功能同上面的自定义的analyzer一样（也可以只实现中文分词），然后再加上其他初始化参数即可

另外，我们发现在sklearn.feature_extraction.text里还有个TfidfVectorizer类，该类继承自CountVectorizer，两者的方法差不多，看名字我们可以知道，它是直接用来计算tfidf的，于是上面计算tf-idf的方法可以用下面这段更简洁的代码

In [7]:
tfidfvectorizer = TfidfVectorizer(analyzer=analyzer)
tfidf = tfidfvectorizer.fit_transform(corpus_zh)

print(tfidfvectorizer.get_feature_names())
print("--------------")
print(tfidfvectorizer.vocabulary_)
print("--------------")
print(tfidf.toarray())

['中国科学院', '北京', '大厦', '天安门', '小明', '来到', '杭研', '毕业', '清华大学', '硕士', '网易']
--------------
{'北京': 1, '毕业': 7, '天安门': 3, '大厦': 2, '来到': 5, '硕士': 9, '杭研': 6, '网易': 10, '中国科学院': 0, '清华大学': 8, '小明': 4}
--------------
[[ 0.          0.52640543  0.          0.          0.          0.52640543
   0.          0.          0.66767854  0.          0.        ]
 [ 0.          0.          0.52547275  0.          0.          0.41428875
   0.52547275  0.          0.          0.          0.52547275]
 [ 0.5         0.          0.          0.          0.5         0.          0.
   0.5         0.          0.5         0.        ]
 [ 0.          0.6191303   0.          0.78528828  0.          0.          0.
   0.          0.          0.          0.        ]]


## 用gensim计算tf-idf<br>
<a href="https://radimrehurek.com/gensim/tutorial.html">gensim</a>也可以计算tf-idf，它不像scikit-learn和英文耦合这么深，参考官方文档可以很快计算tf-idf值